In [20]:
import numpy as np
import pandas as pd
from copy import deepcopy

In [4]:
df = pd.read_csv('data.csv')

In [94]:
tmp_df = df[['tmp10', 'tmp16', 'tmp22']]
wat_df = df[['water_rate', 'water_low', 'water_high']]
tmp_array = tmp_df.values
wat_array = wat_df.values

In [76]:
#check wether temperature is monotonically increasing
center22 = []
center16 = []
for i in enumerate(tmp_array):
    if (i[1][1]-i[1][0])>0 and (i[1][2]-i[1][1])>0:
        center22.append(i[0])
    elif (i[1][1]-i[1][0])>0 and (i[1][1]-i[1][2])>0:
        center16.append(i[0])
    else:
        center22.append(i[0])

In [58]:
from scipy.optimize import curve_fit

In [73]:
def get_gauss(x, y):
    mean = sum(x * y) / sum(y)
    sigma = np.sqrt(sum(y * (x - mean)**2) / sum(y))

    def Gauss(x, a, x0, sigma):
        return a * np.exp(-(x - x0)**2 / (2 * sigma**2))

    popt,pcov = curve_fit(Gauss, x, y, p0=[max(y), mean, sigma])
    return popt

In [119]:
# weighted arithmetic mean (corrected - check the section below)
def get_tmp_para(tmp_array):
    x5 = [10, 16, 22, 28, 34]
    x3 = [10, 16, 22]
    tmp_ans = []
    for i in range(len(tmp_array)):
        y = deepcopy(tmp_array[i])
        if i in center22:
            y = np.append(y, [y[-2], y[-3]])    
            tmp_ans.append(get_gauss(x5, y))
        elif i in center16:
            tmp_ans.append(get_gauss(x3, y))
    return tmp_ans
def get_wat_para(wat_array):
    ans = []
    for i in range(len(wat_array)):
        tmp = np.append(wat_array[i], (wat_array[i][1] + wat_array[i][2])/2)
        y = np.array([tmp[0], tmp[0]*2, tmp[0]])
        x = np.array([tmp[1], tmp[3], tmp[2]])
        ans.append(get_gauss(x, y))
    return ans

In [126]:
wat_para = get_wat_para(wat_array)
tmp_para = get_tmp_para(tmp_array)
all_para = [[*wat_para[i], *tmp_para[i]] for i in range(34)]

In [131]:
from pandas.core.frame import DataFrame

In [137]:
all_para_pandas=DataFrame(all_para)
all_para_pandas.columns = ['wat_1', 'wat_2', 'wat_3', 'tmp_1', 'tmp_2', 'tmp_3']

In [140]:
all_para_pandas.to_csv('gauss_data.csv')